# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
city_weather_df=pd.read_csv("../WeatherPy/city_weather_df.csv")
city_weather_df

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloud coverage,Wind Speed,City datetime
0,Bonavista,CA,48.6499,-53.1147,37.00,100,100,14.97,1617083842
1,La Santisima Trinidad,BO,-14.8333,-64.9000,69.51,100,45,3.94,1617083842
2,Hobart,AU,-42.8794,147.3294,69.80,100,20,11.50,1617083741
3,Verkhoyansk,RU,67.5447,133.3850,16.92,100,78,1.57,1617083843
4,East London,ZA,-33.0153,27.9116,68.00,100,0,9.22,1617083843
...,...,...,...,...,...,...,...,...,...
531,Akçaabat,TR,41.0212,39.5715,50.00,100,40,5.75,1617083968
532,Goderich,CA,43.7501,-81.7165,41.00,100,16,13.80,1617083969
533,Havøysund,NO,70.9963,24.6622,35.60,100,75,4.61,1617083969
534,Tamiahua,MX,21.2667,-97.4500,66.97,100,99,3.11,1617083969


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
#create list for lat and long
humidity = city_weather_df["Humidity"].astype(float)
max_hum=humidity.max()
locations=city_weather_df[["Latitude", "Longitude"]]

In [13]:
#Map using gmaps
fig = gmaps.figure()

#add heat layer to map
heat_layer=gmaps.heatmap_layer(locations,weights=humidity, dissipating=False, 
                                   max_intensity=10, point_radius=1)
# Add layer
fig.add_layer(heat_layer)

#print
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
#Perfect weather conditions, create final df and drop null values 

weather_pref_df = city_weather_df.loc[(city_weather_df["Max Temperature"]>=68) & (city_weather_df["Max Temperature"]<=90)]
wind_pref_df = weather_pref_df.loc[(weather_pref_df["Wind Speed"]<=5)]
cloud_pref_df = wind_pref_df.loc[(wind_pref_df["Cloud coverage"]<5)]
ideal_weather_df = cloud_pref_df.dropna(how = "any")
ideal_weather_df

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloud coverage,Wind Speed,City datetime
96,La Gomera,GT,14.0833,-91.0500,77.00,100,1,4.81,1617083863
313,Miandrivazo,MG,-19.5167,45.4667,81.16,100,0,1.54,1617083915
371,Barra do Garças,BR,-15.8900,-52.2567,69.78,100,2,3.51,1617083931
396,São Mateus,BR,-18.7161,-39.8589,70.97,100,1,4.23,1617083936
422,Susa,IR,32.1942,48.2436,71.01,100,0,1.23,1617083943
488,Conceição da Barra,BR,-18.5933,-39.7322,71.94,100,1,4.76,1617083817
523,Santa Cruz de Barahona,DO,18.2085,-71.1008,74.57,100,1,4.21,1617083889


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
#Create a dataframe called hotel_df to store hotel names along with city, country and coordinates

#store in df 
hotel_df = ideal_weather_df.reset_index(drop=True)

#add hotel name column to df
hotel_df["Hotel Name"]= " "

# geocoordinates
target_radius = 5000
target_type = "hotel"
target_search = "hotel"

params={
    "radius": target_radius, 
    "types" : target_type, 
    "hotel" : target_search, 
    "key" : g_key  
}

for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    long = row["Longitude"]
    
    params["location"]=f"{lat}, {long}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params)
    
    hotel_df = response.json()
    
    print(json.dumps(hotel_df, indent=4, sort_keys=True))


{
    "html_attributions": [],
    "next_page_token": "ATtYBwIY9p35vb--USLZmPguSf2h9IzWWQywj7e8iviPOPK8JYHLK83F_izy7nHuG3a7AOSoFiJbZMZkyTvLZR7zQDxSkT2glMfvRr4QeqSvOFV2CIhHX_qUOvkE0H1OgRD1LFo3x0Yw-QPG6G_sCR1yiBxhIYg-XBlJKC4WxSQ8HvIUOTVZohyzE5nwzmyEB6Zujrf5IbFQTLop_frJ5lfG81QGQ36hdTcX8tiVClBFSf7aJQFiXubzbarh5gGOKxKizLBHbg-xHdLSCpsR3BKRsWrHRJsO9pkGIMbu048RlGHIgQoDzuJeREcVn6476iM14R3O5MxOQcRzqnXihKN2ZbmOCHXL0m7rS0Q5psEivloE4NmdAiW3Fq2-UPovLoL_AYk_0FeudGCPkztRDrdAYTOgCf1GY5R82cPAx98RHcjC6C4joNN2Vw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 14.0833152,
                    "lng": -91.0562193
                },
                "viewport": {
                    "northeast": {
                        "lat": 14.0946231,
                        "lng": -91.0477353
                    },
                    "southwest": {
                        "lat": 14.0767244,
                        "lng": -91.0643006
                    }
   

In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

In [17]:
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]


AttributeError: 'dict' object has no attribute 'iterrows'

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig


# Display figure

Figure(layout=FigureLayout(height='420px'))